<li>
    <p>best_5.pt - yolov8m (ana model V1)</p>
    <p>ortalama tahmin süresi : 0.398</p>
</li> 
<li>
    <p>best_4.pt - yolov8s</p>
    <p>ortalama tahmin süresi : 0.2611756923076923</p>
</li>  
<li>    
    <p>best_3.pt - yolov8s</p>
    <p>ortalama tahmin süresi : 0.3663393846153846</p>
</li>  
<li>
    <p>best_2.pt - yolov8x</p>
    <p>ortalama tahmin süresi : 2.61905846153846</p>
</li>   
<li>
    <p>best_1.pt - yolov8x</p>
    <p>ortalama tahmin süresi : 2.6051687692307692</p>
</li>

In [79]:
import cv2
import numpy as np
from spiral_tools.utils import path_sequence,format_line
from spiral_tools.constants import label_color_dict,label_dict
import os
from ultralytics import YOLO
from datetime import datetime
from IPython.display import clear_output
from sklearn.utils import shuffle
from tqdm import tqdm

## Görsel Tahmin

In [80]:
font = cv2.FONT_HERSHEY_DUPLEX
info_color = (0,0,0)
ds_tag_color = (240,240,255)

save_predicted_frames = False
is_shuffle = True

root = "."
models_path = path_sequence(root,"own_model","models")
test_data_path = ""
test_images_path = ""
test_labels_path = None
images_list = None
labels_list = None

#### Resim Test Tercihi

In [108]:
# model seçimi
print("Hangi model kullanılsın?")
model_names = os.listdir(models_path)
for i,model_name in enumerate(model_names):
    print(f"{i} {model_name}")
c = int(input("\n>"))
clear_output()
if not c >= len(model_names):
    print(f"seçilen model : {model_names[c]}")
    chosen_model_path = path_sequence(root,"own_model","models",model_names[c])
else:
    print("Yanlış Girdi")
    pass
 
model_name = chosen_model_path.split("/")[-1][:-4]
    
# test klasörü seçimi
print("\nHangi türden test olacak?\n0:Test Images klasöründen test (etiketsiz) \n1:Son oluşturulan dataset üzerinden test (etiketli)")
c = int(input("\n>"))
clear_output()
if c == 0:
    test_data_path = path_sequence(root,"own_model","test_images")
    test_images_path = path_sequence(root,"own_model","test_images")
    images_list = os.listdir(test_data_path)
    print("Test Images (etiketsiz seçildi)")
elif c== 1:
    last_ds_name = os.listdir(path_sequence(root,"Dataset_handle","merged",""))[-1]
    test_data_path = path_sequence(root,"Dataset_handle","merged",last_ds_name,"detect")
    test_images_path = path_sequence(test_data_path,"images","test")
    test_labels_path = path_sequence(test_data_path,"labels","test")
    images_list = os.listdir(test_images_path)
    labels_list = os.listdir(test_labels_path)
    print("Son oluşan datasetin test imageleri etiketleriyle rastgele test edilecek.")
else:
    print("Yanlış Girdi")

print("\n",model_name,"\n",f"kaydet: {save_predicted_frames}")

# Ekran çözünürlüğüne ulaşalım
import tkinter as tk
t_ = tk.Tk()

screen_width = t_.winfo_screenwidth()
screen_height = t_.winfo_screenheight()

frame_display_size = (int(screen_width*0.8),int(screen_height*0.8))

#modeli içeri aktaralım
model = YOLO(chosen_model_path)


#rastgelelik
if labels_list != None:
    if is_shuffle:
        images_list,labels_list=shuffle(images_list,labels_list)
else:
    if is_shuffle:
        images_list = shuffle(images_list)
    

Son oluşan datasetin test imageleri etiketleriyle rastgele test edilecek.

 best_5_yolov8 
 kaydet: False


#### işlem

In [115]:
shown_counter = 1

def predict_frame(im):
    start_time = datetime.now()
    results = model.predict(source=im, save=False, save_txt=False,)  # save predictions as labels
    end_time = datetime.now()

    delta = end_time - start_time
    print(" görsel tahmini ",delta.total_seconds()," saniyede tamamlandı")   

    label_list = results[0].boxes.cls.numpy()
    bbox_list = results[0].boxes.xyxy.numpy()
    confidence_list = results[0].boxes.conf.numpy()    
    
    return label_list,bbox_list,confidence_list, delta.total_seconds()



for idx, image_name in tqdm(enumerate(images_list)):
    orig_image = cv2.imread(path_sequence(test_images_path,image_name))
    image = orig_image.copy()
    
    
    # etiketler (eğer varsa)
    if labels_list != None:
        f = open(path_sequence(test_labels_path,labels_list[idx]),"r")
        lines = f.readlines()
        f.close()
        for line in lines:
            label, norm_center_x, norm_center_y, norm_width, norm_height = format_line(line)
            
            width = image.shape[1] * norm_width
            height = image.shape[0] * norm_height
            x1 = int(image.shape[1]*norm_center_x) - int(width/2)
            x2 = int(image.shape[1]*norm_center_x) + int(width/2)
            y1 = int(image.shape[0]*norm_center_y) - int(height/2)
            y2 = int(image.shape[0]*norm_center_y) + int(height/2)

            # etiketleri çizdirelim
            cv2.putText(image,f"{label_dict[int(label)]}",(x1+5,y1+15),font,0.55,ds_tag_color)
            cv2.rectangle(image,(x1,y1),(x2,y2),ds_tag_color,4)
            print(f"{label}:{label_dict[int(label)]}",norm_center_x,norm_center_y,norm_height,norm_height)
        
    # tahminler
    pred_labels, pred_bboxes, pred_confs, pred_time = predict_frame(orig_image)
    if len(pred_bboxes) > 0:
        for lbl ,bbox,conf in zip(pred_labels,pred_bboxes,pred_confs):
            conf = float("{:.3f}".format(conf))
            cv2.rectangle(image,(int(bbox[0]),int(bbox[1])),(int(bbox[2]),int(bbox[3]),),label_color_dict[int(lbl)],2)
            cv2.putText(image,f"{label_dict[int(lbl)]} {conf}",(int(bbox[0]),int(bbox[1])-5),font,0.8,label_color_dict[int(lbl)])
    
    # bilgi yazıları
    cv2.putText(image,str(shown_counter),(10,40),font,1.2,info_color)
    cv2.putText(image,"Sure: {:.3f}".format(pred_time),(10,80),font,1.2,info_color)
    
    
    # göster
    image_ = cv2.resize(image,frame_display_size)
    window_name = f"{image_name}"

    cv2.namedWindow(window_name)
    cv2.moveWindow(window_name,20,20)
    cv2.imshow(window_name,image_)

    key = cv2.waitKey(0)
    if key == ord("q"):
        cv2.destroyAllWindows()
        clear_output()
        break
    elif key == ord("p"):
        clear_output()
        cv2.destroyAllWindows()
        shown_counter+=1 
        cv2.imwrite(path_sequence(root,"own_model","test_predictions",f"{image_name[:-4]}__{model_name}.jpg"),image)
        print("Görüntü Kaydedildi")
        continue
    elif key == ord(" "):
        cv2.destroyAllWindows()
        clear_output()
        shown_counter+=1 

4it [00:06,  1.61s/it]
